In [2]:
# import libraries

import pandas as pd
import tweepy
import os
import json
from datetime import datetime, timedelta
import tweetnlp
import openai

### **Large Test**

In [3]:
# Load the original JSON file

with open("../../Data/Raw/tweets_ws23_v1.json", "r") as file:
        json_data = json.load(file)

def get_all_keys(obj):
    keys = set()
    if isinstance(obj, dict):
        keys.update(obj.keys())
        for value in obj.values():
            keys.update(get_all_keys(value))
    elif isinstance(obj, list):
        for item in obj:
            keys.update(get_all_keys(item))
    return keys

# Get all unique keys
all_keys = get_all_keys(json_data)

# Convert the set to a list to see all unique keys
all_keys_list = list(all_keys)

def flatten_json(y):
    """Flatten a nested JSON object"""
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Flatten the JSON data
flattened_data = [flatten_json(entry) for entry in json_data]

# Convert the list of flattened dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Show the DataFrame
df.head(3)

,source_created_at,author_id,text,source,language,longitude,latitude,id,labels_topic_0_tweet_id,labels_topic_0_user_id,labels_topic_0_relevant,labels_topic_0_topic,labels_topic_0_ground_truth,labels_topic_0_id,labels_sentiment_0_tweet_id,labels_sentiment_0_user_id,labels_sentiment_0_sentiment,labels_sentiment_0_ground_truth,labels_sentiment_0_id,source_id
0,2020-09-18 21:56:20.798000,593731316,@DSisourath The Thameslink core between London...,sprinklr,en,-0.12574,51.50853,acd7673f-e621-5f1a-d662-df278964a6ea,acd7673f-e621-5f1a-d662-df278964a6ea,Z003XDCS,True,service,True,8655353b-cc71-1e89-95d5-1c4e6bf200e1,acd7673f-e621-5f1a-d662-df278964a6ea,Z003XDCS,negative,True,5ee4d218-65a9-70a0-0964-a5b00e594a41,NaN
1,2020-10-13 07:31:53.122000,745583289520496640,@DulwichHistory Loving the complaint about peo...,sprinklr,en,-0.12574,51.50853,5b92aba8-4b05-6c63-8485-e9c870742137,5b92aba8-4b05-6c63-8485-e9c870742137,Z003XDCS,True,delays,True,8aee5c86-6e93-24d4-6d9d-d8a9f2f19cef,5b92aba8-4b05-6c63-8485-e9c870742137,Z003XDCS,negative,True,2f8d741f-59cf-d1ab-27f9-5537a9718f94,NaN
2,2020-10-26 19:27:24.695000,303134761,@SW_Help .And yet you have no toilets on some ...,sprinklr,en,-0.12574,51.50853,0a799c07-8b76-17ba-b840-e538d51e832d,0a799c07-8b76-17ba-b840-e538d51e832d,Z003XDCS,True,toilets,True,2d54f18a-442d-b59b-9daf-0490dc64d21a,0a799c07-8b76-17ba-b840-e538d51e832d,Z003XDCS,negative,True,be479c30-17c3-220f-8eae-eaf6fc874431,NaN


In [10]:
# filter only the columns that we need

df_columns = df[['text','author_id','source_created_at']]
print(df_columns.size)

# remove the duplicated rows
df_columns = df_columns.drop_duplicates(subset=['text'])
df_columns.size

50847


47247

In [11]:
# filter out the tweets that contain the word 'delay' or 'updates' in them

df_filter = df_columns[~df_columns['text'].str.contains('delay', case=False, na=False)]
df_filter = df_filter[~df_filter['text'].str.contains('updates', case=False, na=False)]
df_filter

,text,author_id,source_created_at
0,@DSisourath The Thameslink core between London...,593731316,2020-09-18 21:56:20.798000
1,@DulwichHistory Loving the complaint about peo...,745583289520496640,2020-10-13 07:31:53.122000
2,@SW_Help .And yet you have no toilets on some ...,303134761,2020-10-26 19:27:24.695000
3,@SW_Help you have no toilets on some of your t...,303134761,2020-10-26 19:28:49.281000
4,@SpeedySticks007 @MrNeilJH @TLRailUK @christia...,56427671,2020-09-28 11:59:41.212000
...,...,...,...
16943,At St. Alban's City has the PIS Screen firmwar...,759412872472588288,2019-07-11 15:06:42
16944,Haha oh man the audio corruption on @TLRailUK ...,24940900,2019-07-11 07:34:35
16945,@TLRailUK SweetIs there a plug to charge my ph...,63559912,2020-08-10 11:19:10.181000
16946,@TLRailUK now there are far fewer commuters ha...,277455263,2020-08-29 09:51:10.833000


In [12]:
# add a column with the date and time

df_analysis = df_filter.copy()
df_analysis['date'] = df_analysis['source_created_at'].apply(lambda x: x.split(' ')[0])
df_analysis['time'] = df_analysis['source_created_at'].apply(lambda x: x.split(' ')[1].split('.')[0])
df_analysis.drop(['source_created_at'], axis=1, inplace=True)
df_analysis.head(3)

,text,author_id,date,time
0,@DSisourath The Thameslink core between London...,593731316,2020-09-18,21:56:20
1,@DulwichHistory Loving the complaint about peo...,745583289520496640,2020-10-13,07:31:53
2,@SW_Help .And yet you have no toilets on some ...,303134761,2020-10-26,19:27:24


In [13]:
# filter the latest 50 tweets according to the date and time

df_50 = df_analysis.sort_values(by=['date', 'time'], ascending=False).head(50)

In [14]:
# add a column with the sentiment of the tweet
# keep only the negative and neutral tweets

model = tweetnlp.load_model('sentiment')
df_50['sentiment'] = df_50['text'].apply(lambda x: model.predict(x)['label'])
df_complaints = df_50[(df_50['sentiment'] == 'negative')]
df_complaints.shape

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(19, 5)

### **LLM Analysis**

In [15]:
from keys import OPENAI_API_KEY

OPENAI_API_KEY = OPENAI_API_KEY

In [16]:
# Constructing the prompt for OpenAI
prompt = """
As a senior train maintenance official, I am consolidating passenger complaints for efficient resolution. 
Below are various tweets from train passengers. 
As our customer's opinion is very important to us, analyze each message. 
If it is a complaint, please provide a structured answer. Group the complaints by their topics, and suggest resolutions.
End the reponse with the following line: "Please note that these suggestions are based on the provided information and may need further assessment and customization. Help us improve our service by providing feedback on your experience."

Complaints:
"""

complaint_summary = df_complaints.groupby(['text', 'date']).size().reset_index(name='count')

for index, row in complaint_summary.iterrows():
    prompt += f"- Date Created: {row['date']} - Complaint: {row['text']} (Count: {row['count']})\n"

prompt += """
For each complaint, provide:
1. Summary of the complaint in a very concise manner
2. Topic (1 or 2 words)
3. Suggestion for resolution with actionable steps starting from the next day

Please structure your response as follows:
- Date Created:
- Topic:
- Count of complaints for this topic:
- Summary:
- Suggestion:
"""

# Set up OpenAI API key
openai.api_key = OPENAI_API_KEY

# Generate response from OpenAI
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Replace with the appropriate chat model
    messages=[
        {"role": "system", "content": "You are a senior train maintenance official."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message["content"])
data = response.choices[0].message["content"]

- Date Created: 2020-11-17
- Topic: Train interior design
- Count of complaints for this topic: 1
- Summary: Passenger complains about a pole in the middle of the doors, obstructing wheelchair access.
- Suggestion: Conduct a review of train interior design to ensure unobstructed wheelchair access. Consider removing or repositioning the pole in question. Collaborate with AccessStep and other accessibility experts for guidance.

- Date Created: 2020-11-16
- Topic: Station cleanliness
- Count of complaints for this topic: 1
- Summary: Passenger complains about an unkept area near the station, leading to people dumping rubbish bags.
- Suggestion: Intensify station cleaning efforts, especially in areas prone to rubbish accumulation. Increase surveillance to deter littering. Collaborate with Merton Council to address this issue and improve overall cleanliness.

- Date Created: 2020-11-13
- Topic: Train cleanliness and punctuality
- Count of complaints for this topic: 2
- Summary: Passengers 

In [29]:
# Split the data into separate entries
entries = data.strip().split("- Date Created:")

# Initialize lists to hold the data
dates = []
topics = []
counts = []
summaries = []
suggestions = []

# Process each entry
for entry in entries:
    if entry:  # skip empty strings
        lines = entry.strip().split("\n- ")
        date, topic, count, summary, suggestion = lines
        dates.append(date.replace("Date Created: ", "").strip())
        topics.append(topic.replace("Topic: ", "").strip())
        counts.append(int(count.split(": ")[1]))
        summaries.append(summary.replace("Summary: ", "").strip())
        suggestions.append(suggestion.replace("Suggestion: ", "").strip())

# Create a DataFrame
df = pd.DataFrame({
    'Date of Complaint': dates,
    'Topic': topics,
    'Count': counts,
    'Summary': summaries,
    'Suggestion': suggestions
})

df  # To display the first few rows of the DataFrame

,Date of Complaint,Topic,Count,Summary,Suggestion
0,2020-11-17,Train design,1,Passenger is uncomfortable with the pole in th...,Review the train design to ensure efficient us...
1,2020-11-16,Station cleanliness,1,Passenger complains about the unkept area near...,Increase regular cleaning and maintenance effo...
2,2020-11-06,Noise disturbance,1,Passenger mentions sleepless nights due to noi...,Conduct noise impact assessments and implement...
3,2020-11-06,Train cleanliness and maintenance,2,Passengers express concerns about dirty trains...,Intensify train cleaning and maintenance routi...
4,2020-11-20,Train exterior appearance,1,Passenger suggests that the choice of a light ...,Explore alternative color schemes for future t...
5,2020-11-16,Timetable and platform announcement accuracy,3,Passengers complain about incorrect or confusi...,Improve the accuracy and consistency of timeta...
6,2020-11-23,Communication and service updates,2,Passengers criticize the frequency and accurac...,Enhance communication channels and systems to ...
7,2020-11-21,Face mask requirement,1,Passenger expresses skepticism regarding the e...,Continue to adhere to and enforce the governme...
8,2020-11-23,Train maintenance,1,Passenger reports a broken door on a specific ...,Conduct regular train inspections and maintena...
